In [1]:
DOMAIN = 'cdr' # test, spouse, cdr

In [2]:
import sys
import os

if DOMAIN == 'cdr':
    os.environ['SNORKELDB'] = 'postgres:///semparse_cdr'

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
if DOMAIN in ['test', 'spouse']:
    Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
    candidate_class = Spouse
elif DOMAIN == 'cdr':
    ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])
    candidate_class = ChemicalDisease
else:
    raise Exception("Invalid DOMAIN.")

In [3]:
candidates = session.query(candidate_class).filter(candidate_class.split == 0).all()
print("Candidates: {}".format(len(candidates)))

Candidates: 8272


In [4]:
from snorkel.viewer import SentenceNgramViewer
sv = SentenceNgramViewer(candidates[:300], session)
sv

<IPython.core.display.Javascript object>

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [5]:
c = sv.get_selected()
print c

ChemicalDisease(Span("Cyclosporin A", sentence=9603, chars=[0,12], words=[0,1]), Span("hypertrophy", sentence=9603, chars=[239,249], words=[45,45]))


In [6]:
hash(c)

-8384542269373787667

In [7]:
from semparse_examples import get_examples

examples = get_examples(DOMAIN, candidates)

if DOMAIN == 'test':
    user_lists = {
        'colors':['red','green','blue'],
        'bluebird':['blue','bird','fly'],
        'greek':['alpha','beta','gamma'],
        'letters':['a','B','C'],
        'smalls':['a','b','c','d'],
        'spouse':['wife','husband','spouse']}
elif DOMAIN == 'spouse':
    user_lists = {
        'spouse':['wife','husband','spouse'],
        'family':['father', 'mother', 'brother', 'sister']}
elif DOMAIN == 'cdr':
    user_lists = {}

In [8]:
examples[0].display()

Example
explanation  Label false because arg 1 ends with 'induced'
candidate    ('blue-induced', 'coma')
denotation   -1


In [9]:
from snorkel.semantic import SemanticParser

%time sp = SemanticParser(user_lists, absorb=False)

Created grammar with 265 rules
CPU times: user 13.2 ms, sys: 10.1 ms, total: 23.3 ms
Wall time: 27.9 ms


In [10]:
%time sp.evaluate(examples, debug=True)

Example 0: Label false because arg 1 ends with 'induced'
('.root', ('.label', ('.bool', False), ('.endswith', ('.string', u'i n d u c e d'), ('.arg_to_string', ('.arg', ('.int', 1))))))
('.root', ('.label', ('.bool', False), ('.in', ('.string', u'i n d u c e d'), ('.arg_to_string', ('.arg', ('.int', 1))))))
CPU times: user 22.4 ms, sys: 17.8 ms, total: 40.2 ms
Wall time: 142 ms


,Correct,Incorrect,Redundant,Failed,Unknown
Example0,0,2,0,0,0


In [11]:
# sp.grammar.print_grammar()

In [12]:
# sp.grammar.print_chart()

In [13]:
# total_correct = 0
# total_incorrect = 0
# total_failed = 0
# total_redundant = 0
# for i, example in enumerate(examples):
#     if i==20: break
# #     print example.explanation
#     correct = 0
#     incorrect = 0
#     failed = 0
#     redundant = 0
#     semantics = set()
#     parses = sp.parse(example.explanation, 
#                       example.candidate, 
#                       user_lists=user_lists, 
#                       verbose=False, 
#                       return_parses=True)
# a
#     total_correct += correct
#     total_incorrect += incorrect
#     total_failed += failed
#     total_redundant += redundant
#     valid = correct + incorrect
#     total = correct + incorrect + redundant + failed
#     print "Example %d: %d/%d (%d, %d) -> %d" % (i, correct, valid, redundant, failed, total)
# total_valid = total_correct + total_incorrect
# total_total = total_correct + total_incorrect + total_redundant + total_failed
# print "TOTAL: {}/{} ({:.2f}); {} redundant, {} failed".format(
#                                  total_correct, total_valid, 
#                                  float(total_correct)/total_valid, 
#                                  total_redundant, total_failed)
# print "GROWTH (total): {:.2f}".format(float(total_valid + total_faile)/len(examples))
# print "GROWTH (filter fail): {:.2f}".format(float(total_valid)/len(examples))
# print "GROWTH (filter incorrect): {:.2f}".format(float(total_correct)/len(examples))